In [4]:
from cgra import *
from kernels import *
from sat_to_csv import *
kernel_name = "mmul_os"
convert( kernel_name+"/out.sat", kernel_name+"/instructions.csv" )
# ----------- Op 4 - v2 ---------
# Input data
# 8x8x4
kernel_clear_memory(kernel_name)
# Add B
B = list(range(10, 321, 10))
baseB = 2000
kernel_add_memory_region(kernel_name, baseB, B)
# Add auxiliar address and counters
baseA = 5000
kernel_add_memory_region(kernel_name, baseA, [baseB,baseB+4,baseB+8,baseB+12, 8,2])
# Add A
A = list(range(1, 65, 1))
kernel_add_memory_region(kernel_name, baseA+28, A)
# Set load and write addrs
load_addrs = [baseA, baseA+16, baseA+20, baseA+(4+2+8*3)*4] # Last addrs is 1000+4*(4+2+nColsA*3)
store_addrs = [0, 16, 32, 48] # Each one is one row, so [0, 4*nColsC, 4*nColsC*2, 4*nColsC*3]
run(kernel_name, pr=["ROUT","INST"], load_addrs=load_addrs, store_addrs=store_addrs)

Instr =  0 ( 0 )
[2000,    8,    2,    0]    [LWD R2  4, LWD R1  24, LWD R1  36, NOP ]    
[2004,    0,    0,    0]    [LWD R2  4, NOP , NOP , NOP ]    
[2008,    0,    0,    0]    [LWD R2  4, NOP , NOP , NOP ]    
[2012,    0,    0,    0]    [LWD R2  12, NOP , NOP , NOP ]    
-------
Instr =  1 ( 1 )
[2000,    8,    0,    0]    [NOP , NOP , SADD R2  ZERO  ZERO, NOP ]    
[2004,    0,    0,    0]    [NOP , NOP , NOP , NOP ]    
[2008,    0,    0,    0]    [NOP , NOP , NOP , NOP ]    
[2012,    0,    0,    0]    [NOP , NOP , NOP , NOP ]    
-------
Instr =  2 ( 2 )
[2000,    8,   18,    0]    [NOP , NOP , BEQ R2  R1  18, NOP ]    
[2004,    0,    0,    0]    [NOP , NOP , NOP , NOP ]    
[2008,    0,    0,    0]    [NOP , NOP , NOP , NOP ]    
[2012,    0,    0,    0]    [NOP , NOP , NOP , NOP ]    
-------
Instr =  3 ( 3 )
[   0,    0,    0,    0]    [SADD R3  ZERO  ZERO, SADD R3  ZERO  ZERO, SADD R3  ZERO  ZERO, SADD R3  ZERO  ZERO ]    
[   0,    0,    0,    0]    [SADD R3  ZERO  ZERO

In [9]:
from cgra import *
from kernels import *
kernel_name = "mmul_os"
from sat_to_csv import *
convert( kernel_name+"/out.sat", kernel_name+"/instructions.csv" )

# ----------- Op 4 - v3 ---------
# Input data
# 4x4x4
ROWS_C = 8
COLS_C = 8
COLS_A = 8
# Clear memory
kernel_clear_memory(kernel_name)
# Add A
A = list(range(1, ROWS_C*COLS_A+1, 1))
baseA = 1000
kernel_add_memory_region(kernel_name, baseA, A)
# Add B
B = list(range(1, COLS_A*COLS_C+1, 1))
baseB = 2000
kernel_add_memory_region(kernel_name, baseB, B)
# Add C
baseC = 3000
# Add auxiliar address and counters
# &B[0][0], nItLoopColsC, &A[0][0], &C[0][3], &C[1][0], B[0][1], nItLoopColsA, &A[1][0], ...
nItLoopColsC = int(COLS_C/4)
nItLoopColsA = int(COLS_A)
nItLoopRowsC = int(ROWS_C/4)
baseAux = 0
auxiliar_values = [baseB, nItLoopColsC, baseA, baseC+3*4,
                   baseC+4*COLS_C, baseB+4, nItLoopColsA, baseA+COLS_A*4,
                   baseA+COLS_A*2*4, baseC+4+4*2*COLS_C, baseB+4*2, nItLoopColsC,
                   nItLoopRowsC, baseA+3*4*COLS_A, baseC+8+4*3*COLS_C, baseB+12]
kernel_add_memory_region(kernel_name, baseAux, auxiliar_values)
# Set load and write addrs
load_addrs = [baseAux, baseAux+4*4, baseAux+4*4*2, baseAux+4*4*3]
store_addrs = [baseC+3*4, baseC+4*COLS_C, baseC+4+2*4*COLS_C, baseC+8+3*4*COLS_C]
# Run the kernel
run(kernel_name, limit=1000, pr=["ROUT","R0","R1","R2","R3","INST"], load_addrs=load_addrs, store_addrs=store_addrs)

Instr =  0 ( 0 )
[2000, 3032, 1064,    2]    [   0,    0,    0,    0]    [   0,    0,    0,    2]    [2000, 3032, 1064,    0]    [   0,    0,    0,    0]    [LWD R2  4, LWD R2  4, LWD R2  4, LWD R1  4]    
[   2, 2004, 3068, 1096]    [   0,    0,    0,    0]    [   2,    0,    0,    0]    [   0, 2004, 3068, 1096]    [   0,    0,    0,    0]    [LWD R1  4, LWD R2  4, LWD R2  4, LWD R2  4]    
[1000,    8, 2008, 3104]    [   0,    0,    0,    0]    [   0,    8,    0,    0]    [1000,    0, 2008, 3104]    [   0,    0,    0,    0]    [LWD R2  4, LWD R1  4, LWD R2  4, LWD R2  4]    
[3012, 1032,    2, 2012]    [   0,    0,    0,    0]    [   0,    0,    2,    0]    [3012, 1032,    0, 2012]    [   0,    0,    0,    0]    [LWD R2  4, LWD R2  4, LWD R1  4, LWD R2  4]    
-------
Instr =  1 ( 1 )
[2000, 3032, 1064,    0]    [   0,    0,    0,    0]    [   0,    0,    0,    2]    [2000, 3032, 1064,    0]    [   0,    0,    0,    0]    [NOP , NOP , NOP , SADD R2  ZERO  ZERO]    
[   0, 2004, 3068,